# Langchain crash course

## Restaurant idea generator
CodeBasics Tutorial: LangChain Crash Course For Beginners | LangChain Tutorial
https://www.youtube.com/watch?v=nAmC7SoVLd8

In [1]:
from secret_key import openapi_key
import os
os.environ['OPENAI_API_KEY'] = openapi_key

### LLMs

In [2]:
# from langchain.llms import OpenAI
from langchain_openai import OpenAI
# models: OpenAI, HuggingFace, etc

# temp=0 (most safe) temp=1 (most creative)
llm = OpenAI(temperature=0.6)
# name = llm("I want to open a restaurant for Indian food. Suggest a fancy name for this.")
response = llm.invoke("I want to open a restaurant for Indian food. Suggest a fancy name for this.")
# print(name)

### Prompt Templates

In [3]:
# prompt template
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables = ['cuisine'],
    template = "I want to open a restaurant for {cuisine}. Suggest a fancy name for this."
)
prompt_template_name.format(cuisine="Mexian")

'I want to open a restaurant for Mexian. Suggest a fancy name for this.'

### Chains

In [4]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
# chain.invoke("Italian")
response = chain.invoke("Mexican")
text_output = response['text']
print(response)
print(text_output)

{'cuisine': 'Mexican', 'text': '\n\n"El Dorado Cocina"'}


"El Dorado Cocina"


In [5]:
# Sequential chain

llm = OpenAI(temperature=0.6)

# chain 1 - Generate restaurant name. 
prompt_template_name = PromptTemplate(
    input_variables = ['cuisine'],
    template = "I want to open a restaurant for {cuisine}. Suggest a fancy name for this."
)
name_chain = LLMChain(llm=llm, prompt=prompt_template_name)

# chain 2 - Generate food menu items based on restaurant name
prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template = "Suggest 5 menu items (only name, no description) for {restaurant_name}. Return it as a comma separated list"
)
food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

### Simple Sequential Chain

Gives just one output.   
We can get the menu items based on restaurant, but not get the restaurant name output from this chain.

In [6]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains=[name_chain, food_items_chain])
response = chain.invoke("Mexican")
print(response)
print(response['output'])

{'input': 'Mexican', 'output': '\n\n1. Enchiladas Supremas\n2. Chimichangas\n3. Carne Asada\n4. Fajitas Trio\n5. Tres Leches Cake'}


1. Enchiladas Supremas
2. Chimichangas
3. Carne Asada
4. Fajitas Trio
5. Tres Leches Cake


### Sequential Chain

Gives multiple outputs.  
We can get both the 1) restaurant name and 2) menu items.

In [7]:
from langchain.chains import SequentialChain

llm = OpenAI(temperature=0.6)

# chain 1 - Generate restaurant name. 
prompt_template_name = PromptTemplate(
    input_variables = ['cuisine'],
    template = "I want to open a restaurant for {cuisine}. Suggest a fancy name for this."
)
name_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key='restaurant_name') # added output_key

# chain 2 - Generate food menu items based on restaurant name
prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template = "Suggest 5 menu items (only name, no description) for {restaurant_name}. Return it as a comma separated list"
)
food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items, output_key='menu_items') # added output_key

In [8]:
chain = SequentialChain(
    chains=[name_chain, food_items_chain],
    input_variables=['cuisine'],
    output_variables=['restaurant_name', 'menu_items']
)
response = chain.invoke("Arabic")
print(response)

{'cuisine': 'Arabic', 'restaurant_name': '\n\n"Al-Farah Al-Jameel" (The Beautiful Oasis)', 'menu_items': '\n\n1. Lamb Shawarma\n2. Falafel Platter\n3. Hummus and Pita Bread\n4. Chicken Kebab\n5. Baklava Dessert'}


In [9]:
print(f"Cuisine: {response['cuisine']}")
print(f"Restaurant Name: {response['restaurant_name']}")
print(f"Menu Items: {response['menu_items']}")

Cuisine: Arabic
Restaurant Name: 

"Al-Farah Al-Jameel" (The Beautiful Oasis)
Menu Items: 

1. Lamb Shawarma
2. Falafel Platter
3. Hummus and Pita Bread
4. Chicken Kebab
5. Baklava Dessert


## Additional topics

### Agents

Will use agents to call the tools:
- SerpAPI - Google Search API
- Wikipedia - get information from wikipedia
- llm-math - do math operations

[List of LangChain Tools available](https://python.langchain.com/docs/integrations/tools/)

AgentType: ZERO_SHOT_REACT_DESCRIPTION means agent will have a thought and then reaction to the prompt.

#### Wikipedia and llm-math tool

In [10]:
from langchain.agents import AgentType, create_react_agent, load_tools, initialize_agent
from langchain_openai import OpenAI

tools = load_tools(["wikipedia", "llm-math"], llm=llm)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)
# run agent with verbose output
agent.run("When was Elon Musk born? And what is his age today in 2024?")

c:\ProgramData\miniconda3\envs\langchain_env\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(
c:\ProgramData\miniconda3\envs\langchain_env\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
 I should use Wikipedia to find information about Elon Musk
Action: Wikipedia
Action Input: Elon Musk
Observation: Wikipedia is not a valid tool, try one of [wikipedia, Calculator].
Thought: I should use Wikipedia to find information about Elon Musk's birth date
Action: Wikipedia
Action Input: Elon Musk birth date
Observation: Wikipedia is not a valid tool, try one of [wikipedia, Calculator].
Thought: I should use Wikipedia to find information about Elon Musk's birthdate
Action: Wikipedia
Action Input: Elon Musk birthdate
Observation: Wikipedia is not a valid tool, try one of [wikipedia, Calculator].
Thought: I should use Wikipedia to find information about Elon Musk's birthday
Action: Wikipedia
Action Input: Elon Musk birthday
Observation: Wikipedia is not a valid tool, try one of [wikipedia, Calculator].
Thought: I should use Wikipedia to find information about Elon Musk's personal life
Action: Wikipedia
Action Input: Elon Musk personal life
Ob

'Elon Musk was born on June 28, 1971 and will be 53 years old in 2024.'

#### Serpapi and llm-math tool

In [11]:
from secret_key import serpapi_key
import os
os.environ['SERPAPI_API_KEY'] = serpapi_key

In [12]:
# define LLM
llm = OpenAI(temperature=0.6)

# load tools
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# initialize agent with tools
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)
agent.run("What was the GDP of US in 2022 plus 5?")



> Entering new AgentExecutor chain...
 This is a math question, so I should use the calculator.
Action: Calculator
Action Input: 2022 + 5
Observation: Answer: 2027
Thought: I need to find the GDP of the US in 2027.
Action: Search
Action Input: "US GDP 2027"
Observation: 32,274.46
Thought: This is the final answer I was looking for.
Final Answer: The GDP of the US in 2022 plus 5 is $32,274.46 billion.

> Finished chain.


'The GDP of the US in 2022 plus 5 is $32,274.46 billion.'

### Memory


In [28]:
chain = LLMChain(llm=llm, prompt=prompt_template_name)
response = chain.invoke("Mexican")
print(response['text'])



"El Sabor de México"


In [30]:
type(chain.memory)

NoneType

Our regular chain has no memory of previous interactions.

Now we will use `ConversationBufferMemory` to store the conversation history.  
This can be used to save/export the conversation history.


In [34]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory) # added memory argument

# 1st invocation
response = chain.invoke("Mexican")
print(response['text'])



"El Dorado Cantina"


In [35]:
# 2nd invocation
response = chain.invoke("Italian")
print(response['text'])



"Bella Cucina"


In [36]:
print(type(chain.memory))
print(chain.memory)

<class 'langchain.memory.buffer.ConversationBufferMemory'>
chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Mexican'), AIMessage(content='\n\n"El Dorado Cantina"'), HumanMessage(content='Italian'), AIMessage(content='\n\n"Bella Cucina"')])


In [37]:
print(chain.memory.buffer)

Human: Mexican
AI: 

"El Dorado Cantina"
Human: Italian
AI: 

"Bella Cucina"


This time we see the memory of the whole conversation.  

Next, we will see the memory enable the model to answer questions based on the previous conversation.

In [50]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [51]:
convo.run("Who won the first cricket world cup?")

' The first cricket world cup was held in 1975 and was won by the West Indies team. They defeated the Australia team by 17 runs in the final match. The tournament was held in England and was organized by the International Cricket Council. The first match was played between England and India, with England winning by 202 runs.'

In [52]:
convo.run("How much is 5+5?")

' The sum of 5 and 5 is 10.'

In [53]:
convo.run("Who was the captain ofthe winning team?")

' The captain of the West Indies team during the first cricket world cup was Clive Lloyd. He was known for his strong batting and leadership skills, and he led the team to victory in both the first and second cricket world cups. He was also inducted into the ICC Cricket Hall of Fame in 2003. '

In [54]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup?
AI:  The first cricket world cup was held in 1975 and was won by the West Indies team. They defeated the Australia team by 17 runs in the final match. The tournament was held in England and was organized by the International Cricket Council. The first match was played between England and India, with England winning by 202 runs.
Human: How much is 5+5?
AI:  The sum of 5 and 5 is 10.
Human: Who was the captain ofthe winning team?
AI:  The captain of the West Indies team during the first cricket world cup was Clive Lloyd. He was known for his strong batting and leadership skills, and he led the team to victory in both the first and second cricket world cups. He was also inducted into the ICC Cricket Hall of Fame in 2003. 


We see the LLM was able to answer first about the cricket world cup, then a math question, and finally a follow-up question about the cricket world cup.

However, the memory will just keep filling up and fill the context window with conversation history. 
This large context history might not be neccessary for answering simple questions. It also becomes costly to send soo many tokens through the OpenAI API, which charge per token.
 
Thus, we need to restrict the memory (conversational buffer size) to the last couple of conversational exchanges.   
We use `ConversationalBufferWindowMemory` to restrict the memory to the last couple of conversational exchanges.

In [61]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=2) # keep last 2 conversations

convo = ConversationChain(
    llm=OpenAI(temperature=0.7),
    memory=memory
)

convo.run("Who won the first cricket world cup?")

" The first cricket world cup was won by the West Indies in 1975. The final match was played between the West Indies and Australia at Lord's Cricket Ground in London. The West Indies won by 17 runs and were led by captain Clive Lloyd. The top scorer for the West Indies was Viv Richards with 138 runs. The first cricket world cup was a 60-over tournament, with each team playing six matches in the group stage. The West Indies had a dominant performance throughout the tournament, winning all six of their matches."

In [62]:
convo.run("How much is 5+5?")

'  The result of 5+5 is 10.'

In [63]:
convo.run("Who was the captain ofthe winning team?")

' The captain of the winning team was Clive Lloyd. He was a left-handed batsman and a medium-pace bowler for the West Indies team. He also served as the captain for the West Indies team in the 1979 World Cup. Under his leadership, the West Indies team had a record of 36 wins out of 74 matches. He was known for his aggressive and fearless playing style, and he was inducted into the ICC Cricket Hall of Fame in 2004.'

We seen when using `k=1`, the LLM only remember the last conversational exchange ("How much is 5+5?") and don't remember the previous conversation about the cricket world cup.

With `k=2` the LLM remembers the last two conversational exchanges and is able to answer the follow-up question about the cricket world cup correctly.